In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [86]:
import findspark
from pyspark import  HiveContext , SparkContext

In [87]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [88]:
df_bonus = spark.read.csv('/content/drive/My Drive/Python_Avancé/best_movies_at_home.csv', header=True, inferSchema=True, sep=';')


In [89]:
df_bonus.show(truncate=False)

+-----------------------------------------+---------------------------------------+--------------+---------------+-----------------+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [90]:
#Création  de variables à partir de la chaîne de caractères disponible dans la variable "movie infos"
from pyspark.sql.functions import regexp_extract, col

#Vrai formule : 
result = df_bonus.withColumn('Genre', regexp_extract(col('movie infos'), 'Genre: (.*?)\\t',1)) \
      .withColumn('Trigger Warning', regexp_extract(col('movie infos'), 'Rating: (.*?)\\t',1)) \
      .withColumn('Language', regexp_extract(col('movie infos'), 'Original Language: (.*?)\\t',1))\
      .withColumn('Director', regexp_extract(col('movie infos'), 'Director: (.*?)\\t',1)) \
      .withColumn('Producer', regexp_extract(col('movie infos'), 'Producer: (.*?)\\t',1)) \
      .withColumn('Release date', regexp_extract(col('movie infos'), 'Release Date \(Streaming\): (.*?)\\t',1)) \
      .withColumn('Box Office', regexp_extract(col('movie infos'), 'Box Office \(Gross USA\): (.*?)\\t',1)) \
      .withColumn('Distributor', regexp_extract(col('movie infos'), 'Distributor: (.*?)\\t',1))
# #on peut alors supprimer les caractères spéciaux du CA pour changer la colonne en numérique > réaliser des moyennes après




In [91]:
result.show(truncate=False)

+-----------------------------------------+---------------------------------------+--------------+---------------+-----------------+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [92]:
#On va extraire dans 3 colonnes différentes les informations 

In [93]:
from pyspark.sql import functions as F

infos = F.split(result['infos'], ',')
result = result.withColumn('Release year',infos.getItem(0)) \
               .withColumn('Length',infos.getItem(2)) 

result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+--------------------+------------+-------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|            Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year| Length|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+--------------------+-----

In [94]:
from pyspark.sql.functions import concat, col, lit, split

#Désormais, dans l'objectif de réaliser une étude de la moyenne du temps des films, on va vouloir
#transformer les durées disponibles en un nombre de minutes
#pour faire cela, on va commencer par séparer les heures des minutes

result = result.withColumn('Number of hours', split(result['Length'], 'h').getItem(0)) \
       .withColumn('Number of minutes', split(result['Length'], 'h').getItem(1)) 
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+--------------------+------------+-------+---------------+-----------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|            Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year| Length|Number of hours|Number of minutes|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---

In [95]:
#on va ensuite supprimer le m dans la colonne "Nombre minutes" pour pouvoir transformer cette colonne
#en colonne de type numérique
result = result.withColumn("Number of minutes", F.split(result['Number of minutes'], 'm')[0])
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+--------------------+------------+-------+---------------+-----------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|            Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year| Length|Number of hours|Number of minutes|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---

In [96]:
#on va transformer les colonnes "Nombres heures" et "Nombre minutes" en colonnes numériques
from pyspark.sql.types import IntegerType ,StringType
result = result.withColumn("Number of hours",col("Number of hours").cast(IntegerType())) \
    .withColumn("Number of minutes",col("Number of minutes").cast(IntegerType())) 
result.printSchema()

root
 |-- titles: string (nullable = true)
 |-- infos: string (nullable = true)
 |-- audience state: string (nullable = true)
 |-- audience scores: integer (nullable = true)
 |-- tomatometer state: string (nullable = true)
 |-- tomatometer: integer (nullable = true)
 |-- movie infos: string (nullable = true)
 |-- critics consensus: string (nullable = true)
 |-- critics links: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Trigger Warning: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Producer: string (nullable = true)
 |-- Release date: string (nullable = true)
 |-- Box Office: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- Release year: string (nullable = true)
 |-- Length: string (nullable = true)
 |-- Number of hours: integer (nullable = true)
 |-- Number of minutes: integer (nullable = true)



In [97]:
#on va ensuite multiplier par 60 le nombre d'heures 
result = result.withColumn("Number of hours", 60 * result["Number of hours"] ) 
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+--------------------+------------+-------+---------------+-----------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|            Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year| Length|Number of hours|Number of minutes|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---

In [98]:
#enfin, on va ajouter les deux colonnes "Nombre heures" et "Nombre minutes" pour avoir une colonne
#finale représentant la durée totale de chaque film en minutes
result = result.withColumn("Full length in minutes", result["Number of minutes"] + result["Number of hours"] ) 
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+--------------------+------------+-------+---------------+-----------------+----------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|            Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year| Length|Number of hours|Number of minutes|Full length in minutes|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------

In [99]:
#On peut alors supprimer les colonnes qui ne sont plus nécessaires 
result = result.drop("Length","Number of hours","Number of minutes")
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+--------------------+------------+----------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|            Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year|Full length in minutes|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-------

In [100]:
#Remplacer le tiret par un espace dans la colonne "tomatometer state" & mettre la première lettre en majuscule
#Etape 1 : Remplacer le tiret
from pyspark.sql.functions import regexp_replace
result = result.withColumn('tomatometer state', regexp_replace('tomatometer state', '-', ' ')) 
result.show()


+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+--------------------+------------+----------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|            Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year|Full length in minutes|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-------

In [101]:
#Etape 2 : Mettre en majuscule la première lettre
from pyspark.sql.types import StringType

udf_capitalize  = F.udf(lambda x: str(x).capitalize(), StringType())
result = result.withColumn('tomatometer state', udf_capitalize('tomatometer state')) \
               .withColumn('audience state', udf_capitalize('audience state'))

result.show()
#On dispose alors d'une majuscule dans la nouvelle colonne new_tomatometer_state (avec le changement du tiret intégré)
#On peut alors drop l'ancienne colonne si on le désire

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+--------------------+------------+----------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|            Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year|Full length in minutes|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-------

In [102]:
#Dans la base de données que l'on a scrappé, on observe des différences concernant certains languages qui sont similaires.
#Par exemple, le site différencie "English (United Kingdom)" et "English (Australia)" alors que il s'agit de la même langue.
#On va alors, dans cette étape, créer une nouvelle colonne qui va récupérer uniquement la langue du film pour avoir des résultats plus
#uniformes.
#Pour cela, on va récupérer le premier élément de la colonne Language, en utilisant comme séparateur l'espace.

result = result.withColumn('Language', split(result['Language'], ' ').getItem(0)) 

result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+------------+----------+--------------------+------------+----------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|  Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year|Full length in minutes|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+------------+----------+--------------------+-----

In [103]:
#Sur le site scrappé, un film peut être classé dans plusieurs catégories de genres.
#Nous avons ainsi décidé de récupérer les 3 premiers genres de chaque film, que l'on va classer dans 3 colonnes distinctes pour
#faciliter l'analyse
#Si un film n'a pas plusieurs genres, une valeur manquante sera attribué à ces colonnes. 

genre_extract = F.split(result['Genre'], ',')
result = result.withColumn('Genre n°1',genre_extract.getItem(0)) \
               .withColumn('Genre n°2',genre_extract.getItem(1)) \
               .withColumn('Genre n°3',genre_extract.getItem(2)) \

result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+------------+----------+--------------------+------------+----------------------+------------------+-------------------+-------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|     Trigger Warning|  Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+------

In [104]:
#Puis, dans la colonne TW, on ne va conserver que le classement des films dans les catégories différenciés (PG, R...) pour pouvoir
#faire des comparaisons entre les films lors de l'analyse descriptive de la base de données.
#On va donc, de la même façon que pour la langue, conserver uniquement le premier élément de la colonne.

result = result.withColumn('Trigger Warning', split(result['Trigger Warning'], ' ').getItem(0)) 
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+----------+--------------------+------------+----------------------+------------------+-------------------+-------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+

In [105]:
#Traitement de la colonne box-office > l'objectif est de transformer le type de la colonne en numérique, pour pouvoir calculer
#des moyennes ainsi que des médianes lors de l'analyse descriptive.

#Pour se faire, on va fonctionner en deux étapes 
#Etape 1 : Supprimer les caractères spéciaux ($, M)
#Etape 2 : Changer le type de la colonne en numérique
#Etape 3 : Changer le nom de la colonne pour spécifier malgré tout l'échelle (en millions) et la monnaie (ici le dollar)

In [106]:
#Etape 1 : Supprimer les caractères spéciaux de la colonne "CA du film"

from pyspark.sql.functions import translate
result = result.withColumn('Box Office', translate('Box Office', '$M', '  '))

result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+----------+--------------------+------------+----------------------+------------------+-------------------+-------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+

In [107]:
#Etape 2 : Changer le type de la colonne à l'aide de la fonction DecimalType
from pyspark.sql.types import DecimalType

result = result.withColumn("Box Office",col("Box Office").cast(DecimalType())) 
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+----------+--------------------+------------+----------------------+------------------+-------------------+-------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box Office|         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+

In [108]:
#Etape 3 : Changer le nom de la colonne 
result = result.withColumnRenamed("Box Office","Box-office (in M$) ")
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+-------------------

In [109]:

date_extract = F.split(result['Release date'], ' ')
result = result.withColumn('Release month',date_extract.getItem(0)) \
               .withColumn('Release day',date_extract.getItem(1)) \
               .withColumn('Release year',date_extract.getItem(2)) \

result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+-------------+-----------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release month|Release day|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+---------

In [110]:
result = result.withColumn("Release day", F.split(result['Release day'], ',')[0])
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+-------------+-----------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release month|Release day|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+---------

In [111]:
from pyspark.sql.functions import regexp_replace
result = result.withColumn('Release month', regexp_replace('Release month', 'Jan', '01')) \
               .withColumn('Release month', regexp_replace('Release month', 'Feb', '02')) \
               .withColumn('Release month', regexp_replace('Release month', 'Mar', '03')) \
               .withColumn('Release month', regexp_replace('Release month', 'Apr', '04')) \
               .withColumn('Release month', regexp_replace('Release month', 'May', '05')) \
               .withColumn('Release month', regexp_replace('Release month', 'Jun', '06')) \
               .withColumn('Release month', regexp_replace('Release month', 'Jul', '07')) \
               .withColumn('Release month', regexp_replace('Release month', 'Aug', '08')) \
               .withColumn('Release month', regexp_replace('Release month', 'Sep', '09')) \
               .withColumn('Release month', regexp_replace('Release month', 'Oct', '10')) \
               .withColumn('Release month', regexp_replace('Release month', 'Nov', '11')) \
               .withColumn('Release month', regexp_replace('Release month', 'Dec', '12')) 

result.show(100)

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+-------------+-----------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release month|Release day|
+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+---------

In [112]:
from pyspark.sql import functions as sf
result = result.withColumn('Release date (in date)', 
                    sf.concat(sf.col('Release year'),sf.lit('-'), sf.col('Release month'), sf.lit('-'), sf.col('Release day')))

result.show()


+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+-------------+-----------+----------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release month|Release day|Release date (in date)|
+--------------------+--------------------+--------------+---------------+--------------

In [113]:
from pyspark.sql.types import DateType
result = result.withColumn("Release date (in date)",col("Release date (in date)").cast(DateType())) 
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+-------------+-----------+----------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release month|Release day|Release date (in date)|
+--------------------+--------------------+--------------+---------------+--------------

In [114]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#intégrer une colonne avec la date à laquelle on a récupéré les données
result = result.withColumn("Current date", current_date()-2)
result.show()


+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+-------------+-----------+----------------------+------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release month|Release day|Release date (in date)|Current date|
+--------------------+--------------------+--------------+----

In [115]:
result = result.withColumn('Number of days since the release', datediff(col("current_date"),col("Release date (in date)")))
result.show()

+--------------------+--------------------+--------------+---------------+-----------------+-----------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+--------------------+------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+-------------+-----------+----------------------+------------+--------------------------------+
|              titles|               infos|audience state|audience scores|tomatometer state|tomatometer|         movie infos|   critics consensus|       critics links|               Genre|Trigger Warning|  Language|            Director|            Producer|Release date|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release month|Release day|Release date (in date)|Current date|Number of days since the relea

In [116]:
result = result.drop("infos","movie infos","critics links", "Genre", "Release date", "Release month", "Release day")
result.show()

+--------------------+--------------+---------------+-----------------+-----------+--------------------+---------------+----------+--------------------+--------------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+----------------------+------------+--------------------------------+
|              titles|audience state|audience scores|tomatometer state|tomatometer|   critics consensus|Trigger Warning|  Language|            Director|            Producer|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release date (in date)|Current date|Number of days since the release|
+--------------------+--------------+---------------+-----------------+-----------+--------------------+---------------+----------+--------------------+--------------------+-------------------+--------------------+------------+---------------

In [117]:
result = result.na.fill("Unavailable",["Genre n°1"]) \
               .na.fill("Unavailable",["Trigger Warning"]) \
               .na.fill("Unavailable",["Language"]) \
               .na.fill("Unavailable",["Distributor"]) \
               .na.fill("Unavailable",["Genre n°2"]) \
               .na.fill("Unavailable",["Genre n°3"]) 

result.show()


+--------------------+--------------+---------------+-----------------+-----------+--------------------+---------------+----------+--------------------+--------------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+----------------------+------------+--------------------------------+
|              titles|audience state|audience scores|tomatometer state|tomatometer|   critics consensus|Trigger Warning|  Language|            Director|            Producer|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release date (in date)|Current date|Number of days since the release|
+--------------------+--------------+---------------+-----------------+-----------+--------------------+---------------+----------+--------------------+--------------------+-------------------+--------------------+------------+---------------

In [118]:
result = result.withColumn('audience state', regexp_replace('audience state', 'None', 'Unavailable')) \
               .withColumn('tomatometer state', regexp_replace('tomatometer state', 'None', 'Unavailable')) 
result.show()

+--------------------+--------------+---------------+-----------------+-----------+--------------------+---------------+----------+--------------------+--------------------+-------------------+--------------------+------------+----------------------+------------------+-------------------+-------------------+----------------------+------------+--------------------------------+
|              titles|audience state|audience scores|tomatometer state|tomatometer|   critics consensus|Trigger Warning|  Language|            Director|            Producer|Box-office (in M$) |         Distributor|Release year|Full length in minutes|         Genre n°1|          Genre n°2|          Genre n°3|Release date (in date)|Current date|Number of days since the release|
+--------------------+--------------+---------------+-----------------+-----------+--------------------+---------------+----------+--------------------+--------------------+-------------------+--------------------+------------+---------------

In [119]:
result_pandas = result.toPandas()
result_pandas

,titles,audience state,audience scores,tomatometer state,tomatometer,critics consensus,Trigger Warning,Language,Director,Producer,Box-office (in M$),Distributor,Release year,Full length in minutes,Genre n°1,Genre n°2,Genre n°3,Release date (in date),Current date,Number of days since the release
0,PUSS IN BOOTS: THE LAST WISH,Upright,93.0,Certified fresh,95.0,Arriving more than a decade after the previous...,PG,English,Joel Crawford,Mark Swift,112,Universal Pictures,2023,102,Kids & Family,Comedy,Adventure,2023-01-06,2023-01-27,23.0
1,THE MENU,Upright,76.0,Certified fresh,88.0,While its social commentary relies on basic in...,R,English,Mark Mylod,"Adam McKay, Betsy Koch, Will Ferrell",38,Searchlight Pictures,2023,107,Horror,Mystery & Thriller,Comedy,2023-01-03,2023-01-27,26.0
2,THE BANSHEES OF INISHERIN,Upright,76.0,Certified fresh,97.0,Featuring some of Martin McDonagh's finest wor...,R,English,Martin McDonagh,"Graham Broadbent, Peter Czernin, Martin McDonagh",9,Searchlight Pictures,2022,109,Comedy,Unavailable,Unavailable,2022-12-13,2023-01-27,47.0
3,THE PALE BLUE EYE,Upright,72.0,Fresh,62.0,The Pale Blue Eye lacks its source material's ...,R,English,Scott Cooper,"Christian Bale, Scott Cooper, John Lesher, Tyl...",None,Netflix,2023,130,Mystery & Thriller,Drama,Unavailable,2023-01-06,2023-01-27,23.0
4,EVERYTHING EVERYWHERE ALL AT ONCE,Upright,89.0,Certified fresh,95.0,"Led by an outstanding Michelle Yeoh, Everythin...",R,English,"Dan Kwan, Daniel Scheinert","Joe Russo, Anthony Russo, Mike Larocca, Dan Kw...",70,A24,2022,132,Comedy,Adventure,Sci-Fi,2022-06-07,2023-01-27,236.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,AMERICAN PSYCHO,Upright,85.0,Fresh,68.0,If it falls short of the deadly satire of Bret...,R,English,Mary Harron,"Christian Halsey Solomon, Chris Hanley, Edward...",15,Lionsgate Films,2005,103,Comedy,Mystery & Thriller,Horror,2005-06-21,2023-01-27,6431.0
146,PARASITE,Upright,90.0,Certified fresh,99.0,"An urgent, brilliantly layered look at timely ...",R,Korean,Bong Joon Ho,"Kwak Sin-ae, Moon Yanggwon",53,Neon,2019,132,Comedy,Mystery & Thriller,Drama,2019-10-11,2023-01-27,1206.0
147,PADDINGTON 2,Upright,88.0,Certified fresh,99.0,Paddington 2 honors its star's rich legacy wit...,PG,English,Paul King,David Heyman,38,Warner Bros. Pictures,2018,103,Kids & Family,Comedy,Adventure,2018-04-24,2023-01-27,1741.0
148,CAUSEWAY,Upright,74.0,Certified fresh,86.0,Causeway takes a powerfully subdued look at th...,R,English,Lila Neugebauer,"Jennifer Lawrence, Justine Polsky",None,A24,2022,92,Drama,Unavailable,Unavailable,2022-11-04,2023-01-27,86.0


In [121]:
result_pandas.to_csv('/content/drive/My Drive/Python_Avancé/best_movies_at_home_finale.csv', sep=";", index=False)